# Valuation of Housing Developments in SF

### Imports

In [ ]:
# Importing Necessary Packages

import pandas as pd
import numpy as np
import urllib.request
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import reverse_geocoder as rg
import pprint
import geocoder
import multiprocessing as mp
import geopy
from geopy.geocoders import Nominatim
from geopy.point import Point
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingRegressor

## Table of Contents

### 1. Exploratory Data Analysis and Cleaning
####    a. Valuations Dataset
####    b. Police Dataset
####    c. City Survey Dataset
####    d. Final Dataset Join
### 2. Baseline Model (Linear Regression)
####    a. Prep/Split/Preprocessing
####    b. Model Definition/Run Model
####    c. Model Results
### 3. Final Model (Random Forest)
####    a. Prep/Split/Preprocessing
####    b. Model Definition/Run Model
####    c. Model Results

## 1. Exploratory Data Analysis and Cleaning

### 1a. Valuations Dataset

In [ ]:
# Read in valuation data
valuation = pd.read_csv("Assessor_Historical_Secured_Property_Tax_Rolls.csv", low_memory=False)

In [ ]:
# Drop nan values in geom data
val2 = valuation.dropna(subset = ['the_geom'])
val2.the_geom

In [ ]:
# Format geom data for our use
val2["x"] = val2["the_geom"].str.split("\s+").str[1].str.replace("\(", "")
val2["y"] = val2["the_geom"].str.split("\s+").str[2].str.rstrip(")")

# Convert extracted values to numeric type
val2["x"] = pd.to_numeric(val2["x"])
val2["y"] = pd.to_numeric(val2["y"])
val2["x"] 

In [ ]:
# Read in Zip GeoDF
data_poly = gpd.read_file("San Francisco ZIP Codes.geojson")

In [ ]:
# Convert val2 to GeoDF and join with Zip GeoDF

gdf = gpd.GeoDataFrame(val2, geometry=gpd.points_from_xy(val2.x, val2.y))
joined_gdf = gpd.sjoin(gdf, data_poly, op='within')

In [ ]:
# Display Columns in Joined DF
joined_gdf.columns

In [ ]:
# Choose Relevant Columns
val_clean = joined_gdf[["Closed Roll Year", "id", "Assessed Improvement Value", "Assessed Land Value", "Assessed Personal Property Value",
"Year Property Built", "Number of Bathrooms","Number of Bedrooms", "Number of Rooms", "Number of Stories", "Number of Units", "Assessor Neighborhood District", "Assessor Neighborhood Code"]]

In [ ]:
# Keep only data from the years 2013 through 2019
val_clean = val_clean[(val_clean["Closed Roll Year"] >= 2013) & (val_clean["Closed Roll Year"] <= 2019)]

# Create Total Assesed Value which equals Assessed Personal Property Value + Assessed Improvement Value
val_clean["Total Assessed Value"] = val_clean["Assessed Land Value"] + val_clean["Assessed Improvement Value"]
val_clean = val_clean.rename(columns = {"id":"Zipcode"})

In [ ]:
# Check for invalid zipcodes (none)
val_clean["Zipcode"].value_counts()

In [ ]:
# Check to look at Total Assessed Value
val_clean["Total Assessed Value"].value_counts()

In [ ]:
# Drop the rows where Total Assessed Value = 0
val_clean = val_clean[val_clean["Total Assessed Value"] != 0]
val_clean["Total Assessed Value"].value_counts()

In [ ]:
# Take a precursory look at val_clean
val_clean.describe()

In [ ]:
# Take a look at Distribution of Total Assessed Values
plt.hist(val_clean['Total Assessed Value'], bins=50)
plt.xlabel('Total Assessed Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Take a look at how year built may affect Total Assessed Value
sns.scatterplot(x='Year Property Built', y='Total Assessed Value', data=val_clean)
plt.xlabel('Year Property Built')
plt.ylabel('Total Assessed Value')
plt.show()

In [ ]:
# Take a look at how many properties are in each assessment district
plt.figure(figsize=(12,6))
sns.countplot(x='Assessor Neighborhood District', data=val_clean)
plt.xlabel('Assessor Neighborhood District')
plt.ylabel('Number of Properties')
plt.show()

In [ ]:
# Mapping property values by zip code
map_sf = folium.Map(location=[37.7749, -122.4194], zoom_start=12)
choropleth = folium.Choropleth(geo_data='San Francisco ZIP Codes.geojson', name='choropleth', data=val_clean, columns=['Zipcode', 'Total Assessed Value'], key_on='feature.properties.zipcode', fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2, legend_name='Total Assessed Value').add_to(map_sf)
folium.LayerControl().add_to(map_sf)
map_sf

### 1b. Police Dataset

In [ ]:
# Read in SFPD Crime Datasets

police_df1318 = pd.read_csv("PoliceDepartment_IncidentReports_2003_May2018.csv")
police_df_18 = pd.read_csv("Police_Department_Incident_Reports__2018_to_Present.csv")

In [ ]:
# Create geometry columns in order to extract zipcode later
police_df_18['geometry'] = police_df_18['Point']
police_df1318['geometry'] = police_df1318['location']

In [ ]:
# Drop unneeded columns
police_df13 = police_df1318.drop(columns=['DELETE - 2017 Fix It Zones 2 2', 'Civic Center Harm Reduction Project Boundary 2 2',
          'Fix It Zones as of 2017-11-06  2 2', 'DELETE - HSOC Zones 2 2', "location",
          'Fix It Zones as of 2018-02-07 2 2', 'CBD, BID and GBD Boundaries as of 2017 2 2', 
          'Central Market/Tenderloin Boundary 2 2', 'Central Market/Tenderloin Boundary Polygon - Updated 2 2',
          'HSOC Zones as of 2018-06-05 2 2', 'OWED Public Spaces 2 2', 'DELETE - Fire Prevention Districts 2 2','DELETE - Police Districts 2 2',
          'DELETE - Supervisor Districts 2 2','DELETE - Zip Codes 2 2','DELETE - Neighborhoods 2 2',
          'SF Find Neighborhoods 2 2', 'Current Police Districts 2 2','Time',
          'Current Supervisor Districts 2 2','Analysis Neighborhoods 2 2', 'DayOfWeek', 'Resolution',
          'Areas of Vulnerability, 2016 2 2','Neighborhoods 2', 'Incident Code', 'Descript'])

police_df18 = police_df_18.drop(columns=['Report Datetime', 'Row ID','Incident Subcategory', 'Incident Description',
        'Resolution', 'Intersection', 'CNN', 
        'Analysis Neighborhood', 'Supervisor District','Point',
        'Supervisor District 2012', 'CAD Number',
        'Report Type Code', 'Report Type Description', 'Filed Online',
        'Incident Code',
        'Neighborhoods', 'ESNCAG - Boundary File',
        'Central Market/Tenderloin Boundary Polygon - Updated','Incident Day of Week', 
        'Civic Center Harm Reduction Project Boundary', 'Incident Date', 'Incident Time', 'Incident Year',
        'HSOC Zones as of 2018-06-05', 'Invest In Neighborhoods (IIN) Areas',
        'Current Supervisor Districts', 'Current Police Districts'])

In [ ]:
# Format 2018-Present to prepare for combining the two separate sets.
police_df18 = police_df18.rename(columns = {'Incident ID':'PdId', 'Incident Number': 'IncidntNum', 'Longitude':'X', 'Latitude':'Y',
                                           'Incident Datetime':'Date', 'Police District':'PdDistrict', 'Incident Category': 'Category',
                                           'geometry':'geometry'})

# Placeholder for address for 2018-present data
police_df18['Address'] = 0

In [ ]:
# Double-check whether columns are the same across 2013-2018 & 2018-Present dataset
police_df13.columns, police_df18.columns #True

In [ ]:
# Append dataframes
police_df18 = police_df18[['PdId', 'IncidntNum', 'Category', 'Date', 'PdDistrict', 'Address','X', 'Y', 'geometry']]
police_df13.columns == police_df18.columns
police = pd.concat([police_df18, police_df13])

In [ ]:
# Filter out overlap for 2018 & any other duplicates
police = police.drop_duplicates()
police.duplicated().unique()

In [ ]:
# Convert Date column to datetime
police['Incident Date'] = pd.to_datetime(police['Date'])
police['Year'] = police['Incident Date'].dt.year # create a new Year column
police

In [ ]:
# Filter for years 2013-2019
police_filtered = police.loc[police['Year'] <= 2019]
police_filtered = police_filtered.loc[police_filtered['Year'] >= 2013]
police_filtered

In [ ]:
# Take a look at number of incidents per year

sns.set_style('darkgrid')
plt.figure(figsize=(10, 6))
sns.countplot(x='Year', data=police_filtered)
plt.title('Incidents by Year')
plt.show()

In [ ]:
# Take a look at number of incidents by police district

plt.figure(figsize=(12, 8))
sns.countplot(y='PdDistrict', data=police_filtered, order=police_filtered['PdDistrict'].value_counts().index)
plt.title('Incidents by Police District')
plt.show()

In [ ]:
# Let's map the incidents

plt.figure(figsize=(12, 8))
scatterplot = sns.scatterplot(x='X', y='Y', hue='Category', data=police_filtered, alpha=0.5, legend='brief')
scatterplot.legend(loc='center right', bbox_to_anchor=(1.3, 0.5), ncol=1)
plt.title('Incident Category by Location')
plt.show()

In [ ]:
# Drop all missing coordinate values
police_filtered = police_filtered.dropna(subset=['X','Y'], how='all')
police_filtered.head()

In [ ]:
# Convert latitude and longitude to zipcode
data_poly = gpd.read_file("San Francisco ZIP Codes.geojson")

In [ ]:
# Create Police GeoDF and join with Zip Code GDF
gdf = gpd.GeoDataFrame(police_filtered, geometry=gpd.points_from_xy(police_filtered.X, police_filtered.Y))
joined_gdf = gpd.sjoin(gdf, data_poly, op='within')

In [ ]:
# Take a look at the count of instances per zip code
joined_gdf['id'].value_counts()

In [ ]:
# Choose relevant columns and get info
police_final = joined_gdf[['PdId', 'IncidntNum', 'Category', 'Date', 'Year', 'id']].rename(columns = {'id': 'zipcode'})
police_final.info()

### 1c. City Survey Dataset

In [ ]:
# Read in City Survey data
xls = pd.ExcelFile('City Survey Master Data 1996-2019.xlsx')
city_df = pd.read_excel(xls, 'Historical Data')

In [ ]:
# Isolate columns of interest
columns = [ "year", "dem_zip", "dem_district", "inf_clean", "inf_stcond", "inf_sidecond", "inf_clean_side", "inf_clean_st", "safe_day", "safe_night", "dem_hhsize"]
dfagg = city_df[columns]

In [ ]:
# Isolate columns for cleanliness & exclude missing values/columns with ratings of 6 or 7
relevant_columns = ["inf_clean", "inf_stcond", "inf_sidecond", "inf_clean_side", "inf_clean_st"]
rating_map = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
}
dfagg[relevant_columns] = dfagg[relevant_columns].replace(rating_map)

In [ ]:
# Convert the ratings to numeric values and replace non-integer values with NaN
dfagg[relevant_columns] = dfagg[relevant_columns].applymap(lambda x: pd.to_numeric(x, errors='coerce'))

In [ ]:
# Compute the mean for each row, excluding NaN values
dfagg['cleanliness'] = dfagg[relevant_columns].mean(axis=1, skipna=True)

In [ ]:
# Print the result
dfagg['cleanliness'].value_counts()

In [ ]:
# Create a cleaned df for 2019
dfagg = dfagg.drop(relevant_columns, axis=1)
df2019_cleaned = dfagg.rename(columns={"dem_zip":"zipcode", "dem_district": "district","dem_hhsize":"household_size"})
df2019_cleaned

In [ ]:
# Find entries without/invalid zipcodes & replace with 0
df2019_cleaned['zipcode'] = df2019_cleaned['zipcode'].fillna(0).astype(int)
df2019_cleaned['zipcode'] = df2019_cleaned['zipcode'].replace([88888, 99999], 0)

In [ ]:
# Filter for years 2013-2019
survey_filtered = df2019_cleaned.loc[df2019_cleaned['year'] <= 2019]
survey_filtered = survey_filtered.loc[df2019_cleaned['year'] >= 2013]
survey_filtered

# Notes: No zipcodes available for 2015, Only have data every other year starting 2005

In [ ]:
# Group the data by year and zipcode and count the occurrences
grouped = survey_filtered.groupby(['year', 'zipcode']).size().unstack()

In [ ]:
# Plot occurences by year and zipcode
ax = grouped.plot(kind='bar', stacked=True, figsize=(10, 6))

ax.set_xlabel('Year') 
ax.set_ylabel('Count')
ax.legend(title='Zipcode', loc='upper right')
ax.set_title('Distribution of Zipcodes by Year')
plt.show()

In [ ]:
# Take a look at count of responses by year
sns.set_style('darkgrid')
plt.figure(figsize=(10, 6))
sns.countplot(x='year', data=survey_filtered)
plt.title('Number of Responses by Year')
plt.show()

In [ ]:
# 2015 has no zipcode data
survey_filtered[survey_filtered['year'] == 2015]

In [ ]:
# Unable to identify correct zipcode for zipcode value 0 (missing data) by district since there are multiiple zipcodes per district
survey_filtered.groupby(['district', 'zipcode']).count()

In [ ]:
# Fill in gap years with previous year data (i.e. fill in 2014 & 2015 & 2016 data with 2013 data)

# Drop 2015 data without zipcodes
survey_filtered1 = survey_filtered[survey_filtered.zipcode != 0]
survey_filtered1.year.value_counts()

# Want 2013-2019

# Create 2014, 2015, 2016 data with 2013 data
df2014 = survey_filtered1[survey_filtered1.year == 2013]
df2014['year'] = df2014['year'].replace(2013, 2014)
df2015 = survey_filtered1[survey_filtered1.year == 2013]
df2015['year'] = df2015['year'].replace(2013, 2015)
df2016 = survey_filtered1[survey_filtered1.year == 2013]
df2016['year'] = df2016['year'].replace(2013, 2016)

# Create 2018 data with 2017 data
df2018 = survey_filtered1[survey_filtered1.year == 2017]
df2018['year'] = df2018['year'].replace(2017, 2018)

# Append all dataframes back to original df --> survey_filtered
survey_final = pd.concat([survey_filtered1, df2014, df2015, df2016, df2018])
survey_final

In [ ]:
# Group the data by zip and day
grouped = survey_final.groupby(['zipcode', 'safe_day']).size().unstack()

# Plot a stacked bar chart
ax = grouped.plot(kind='bar', stacked=True, figsize=(10, 6))
ax.set_xlabel('Zipcode') 
ax.set_ylabel('Count')
ax.legend(title='Safety Rating (5: Highest, 1: Lowest)', loc='upper right')
ax.set_title('Distribution of Safety During Day by Zipcode')
plt.show()

In [ ]:
# Group the data by zip and night
grouped = survey_final.groupby(['zipcode', 'safe_night']).size().unstack()

# Plot a stacked bar chart
ax = grouped.plot(kind='bar', stacked=True, figsize=(10, 6))
ax.set_xlabel('Zipcode') 
ax.set_ylabel('Count')
ax.legend(title='Safety Rating (5: Highest, 1: Lowest)', loc='upper right')
ax.set_title('Distribution of Safety During Night by Zipcode')
plt.show()

### 1d. Final Dataset Join

In [ ]:
# Final dataframes for Years 2013-2019 (will join with valuation data)
display(survey_final)
display(police_final)

In [ ]:
# Format valuation data in order to join
val_final = val_clean.rename(columns = {'Closed Roll Year': 'Year'})
val_final.head()

In [ ]:
# Get the count of incidents from police_final 
police_final2 = police_final.groupby(['Year','zipcode']).count().reset_index().rename(columns = {'PdId':'Incident Counts', 'zipcode':'Zipcode'})
police_final2 = police_final2.drop(columns = ['IncidntNum','Date', 'Category'])
police_final2

In [ ]:
# Get average of safe_day,safe_night,household_size,cleanliness for each year,zipcode pair
survey_final2 = survey_final.groupby(['year','zipcode']).mean().reset_index().drop(columns = ['district']).rename(columns = {'year': 'Year', 'zipcode':'Zipcode'})
survey_final2

In [ ]:
# Check datatypes
#val_final.info(), police_final2.info(), survey_final2.info()

# Convert police_final2 Zipcode column from str-->int
police_final2['Zipcode'] = police_final2.Zipcode.astype(int)
police_final2

val_final.info(), police_final2.info(), survey_final2.info()

In [ ]:
# Left join on val_final with police_final2
join1 = pd.merge(val_final, police_final2, on = ['Year', 'Zipcode'], how = 'left')
join1

In [ ]:
# Drop nan values
join1 = join1.dropna(subset = ['Year Property Built'])
join1

In [ ]:
# Left join join1 with survey_final2
final_df = pd.merge(join1, survey_final2, on = ['Year', 'Zipcode'], how = 'inner')

In [ ]:
# Correlation of Total Assessed Value with other features
final_df.corr().loc['Total Assessed Value']

# Drop Assessed Improvement Value & Assessed Land Value since both were used to calculate to Total Assessed Value
final_df = final_df.drop(columns = ['Assessed Improvement Value', 'Assessed Land Value'])

# Final Dataframe
final_df

In [ ]:
# One hot encode zipcode
df_encoded = pd.get_dummies(final_df, columns = ['Zipcode'], drop_first = True) #avoid multicollinearity
df_encoded.columns

## 2. Baseline Model (Linear Regression)

*Note: Model might underestimate since increase in tax rate does not keep up with real-world inflation in California.*

### 2a. Prep/Split/Preprocessing

In [ ]:
# Check for nan values
df_encoded.isna().sum()

df_encoded = df_encoded.dropna()
df_encoded.columns


# Log of Total Assessed Value
df_encoded['Log of Total Assessed Value'] = np.log(df_encoded['Total Assessed Value'])

In [ ]:
# train_test split
X = df_encoded[['Year', 'Assessed Personal Property Value', 'Year Property Built',
       'Number of Bathrooms', 'Number of Bedrooms', 'Number of Rooms',
       'Number of Stories', 'Number of Units', 
       'Incident Counts', 'safe_day', 'safe_night', 'household_size',
       'cleanliness', 'Zipcode_94103', 'Zipcode_94104', 'Zipcode_94105',
       'Zipcode_94107', 'Zipcode_94108', 'Zipcode_94109', 'Zipcode_94110',
       'Zipcode_94111', 'Zipcode_94112', 'Zipcode_94114', 'Zipcode_94115',
       'Zipcode_94116', 'Zipcode_94117', 'Zipcode_94118', 'Zipcode_94121',
       'Zipcode_94122', 'Zipcode_94123', 'Zipcode_94124', 'Zipcode_94127',
       'Zipcode_94131', 'Zipcode_94132', 'Zipcode_94133', 'Zipcode_94134',
       'Zipcode_94158']]
Y = df_encoded['Log of Total Assessed Value']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .3, random_state = 42)

### 2b. Model Definition/Run Model

In [ ]:
# Model definition
model = LinearRegression()
model.fit(X_train, Y_train)

In [ ]:
# Construct dataframe of each feature & the corresponding coefficients
feature = ['Year', 'Assessed Personal Property Value', 'Year Property Built',
       'Number of Bathrooms', 'Number of Bedrooms', 'Number of Rooms',
       'Number of Stories', 'Number of Units', 
       'Incident Counts', 'safe_day', 'safe_night', 'household_size',
       'cleanliness', 'Zipcode_94103', 'Zipcode_94104', 'Zipcode_94105',
       'Zipcode_94107', 'Zipcode_94108', 'Zipcode_94109', 'Zipcode_94110',
       'Zipcode_94111', 'Zipcode_94112', 'Zipcode_94114', 'Zipcode_94115',
       'Zipcode_94116', 'Zipcode_94117', 'Zipcode_94118', 'Zipcode_94121',
       'Zipcode_94122', 'Zipcode_94123', 'Zipcode_94124', 'Zipcode_94127',
       'Zipcode_94131', 'Zipcode_94132', 'Zipcode_94133', 'Zipcode_94134',
       'Zipcode_94158']

pd.DataFrame(zip(feature, model.coef_), columns = ['feature', 'coefficients'])

# Note: Zipcode_94104 & few others have many commercial buildings / tech companies

In [ ]:
Y_pred_test = model.predict(X_test)
Y_pred_train = model.predict(X_train)
mean_squared_error(np.exp(Y_test), np.exp(Y_pred_test))**(1/2), mean_squared_error(np.exp(Y_train), np.exp(Y_pred_train))**(1/2)

**Hyperparameter Tuning for Linear Regression**

In [ ]:
# Narrow down features to the ones most correlated with Log of Total Assessed Value
df_encoded.corr().loc['Total Assessed Value'].sort_values(ascending=False)

In [ ]:
# Take all features with >0.05 or <-0.05 coefficient & redo train-test split

# train_test split
X = df_encoded[['Number of Bathrooms', 'Number of Rooms', 'Zipcode_94123','Number of Units','Zipcode_94105','Zipcode_94124','household_size'                    
,'Zipcode_94112','Zipcode_94134','Zipcode_94122','Zipcode_94116','cleanliness','Year','Year Property Built','Zipcode_94104'                       
,'Zipcode_94115','Number of Bedrooms','Zipcode_94109','Zipcode_94111','Number of Stories','Zipcode_94114','Zipcode_94103'                     
,'Zipcode_94108','safe_day']]
Y = df_encoded['Log of Total Assessed Value']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .3, random_state = 42)

In [ ]:
# Model Definition
model2 = LinearRegression()
model2.fit(X_train, Y_train)

In [ ]:
feature2 = ['Number of Bathrooms', 'Number of Rooms', 'Zipcode_94123','Number of Units','Zipcode_94105','Zipcode_94124','household_size'                    
,'Zipcode_94112','Zipcode_94134','Zipcode_94122','Zipcode_94116','cleanliness','Year','Year Property Built','Zipcode_94104'                       
,'Zipcode_94115','Number of Bedrooms','Zipcode_94109','Zipcode_94111','Number of Stories','Zipcode_94114','Zipcode_94103'                     
,'Zipcode_94108','safe_day']

# Construct dataframe of each feature & the corresponding coefficients
pd.DataFrame(zip(feature, model2.coef_), columns = ['feature', 'coefficients'])

In [ ]:
# Predict on train set
Y_pred_train = model2.predict(X_train)

# RMSE
mean_squared_error(Y_train, Y_pred_train)**(1/2)

In [ ]:
# Predict on test set
Y_pred_test = model2.predict(X_test)

# RMSE
mean_squared_error(Y_test, Y_pred_test)**(1/2)

### 2c. Model Results

## 3. Final Model (Random Forest)

### 3a. Prep/Split/Preprocessing

In [ ]:
# train_test split
X = df_encoded[['Year', 'Assessed Personal Property Value', 'Year Property Built',
       'Number of Bathrooms', 'Number of Bedrooms', 'Number of Rooms',
       'Number of Stories', 'Number of Units', 
       'Incident Counts', 'safe_day', 'safe_night', 'household_size',
       'cleanliness', 'Zipcode_94103', 'Zipcode_94104', 'Zipcode_94105',
       'Zipcode_94107', 'Zipcode_94108', 'Zipcode_94109', 'Zipcode_94110',
       'Zipcode_94111', 'Zipcode_94112', 'Zipcode_94114', 'Zipcode_94115',
       'Zipcode_94116', 'Zipcode_94117', 'Zipcode_94118', 'Zipcode_94121',
       'Zipcode_94122', 'Zipcode_94123', 'Zipcode_94124', 'Zipcode_94127',
       'Zipcode_94131', 'Zipcode_94132', 'Zipcode_94133', 'Zipcode_94134',
       'Zipcode_94158']]
Y = df_encoded['Log of Total Assessed Value']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .3, random_state = 42)

### 3b. Model Definition/Run Model

In [ ]:
# Define Model
rf = RandomForestRegressor(n_estimators=10, random_state=42)  
rf.fit(X_train, Y_train)

In [ ]:
# Make predictions on test
Y_pred2 = rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(Y_test, Y_pred2)
rmse = mse ** (1/2)

print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")

***Log Features & Response***

In [ ]:
#safe_day, safe_night, cleanliness --> convert these to binary (<3 not safe/clean, >= 3 safe/clean)
df_encoded['safe_day_binary'] = np.where(df_encoded['safe_day']<3, 0, 1)
df_encoded['safe_night_binary'] = np.where(df_encoded['safe_night']<3, 0, 1)
df_encoded['cleanliness_binary'] = np.where(df_encoded['cleanliness']<3, 0, 1)

In [ ]:
X = df_encoded[['Year', 'Assessed Personal Property Value', 'Year Property Built',
       'Number of Bathrooms', 'Number of Bedrooms', 'Number of Rooms',
       'Number of Stories', 'Number of Units', 
       'Incident Counts', 'safe_day_binary', 'safe_night_binary', 'household_size',
       'cleanliness_binary', 'Zipcode_94103', 'Zipcode_94104', 'Zipcode_94105',
       'Zipcode_94107', 'Zipcode_94108', 'Zipcode_94109', 'Zipcode_94110',
       'Zipcode_94111', 'Zipcode_94112', 'Zipcode_94114', 'Zipcode_94115',
       'Zipcode_94116', 'Zipcode_94117', 'Zipcode_94118', 'Zipcode_94121',
       'Zipcode_94122', 'Zipcode_94123', 'Zipcode_94124', 'Zipcode_94127',
       'Zipcode_94131', 'Zipcode_94132', 'Zipcode_94133', 'Zipcode_94134',
       'Zipcode_94158']]
Y = df_encoded['Log of Total Assessed Value']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .3, random_state = 42)

In [ ]:
# Re-define Model 
rf = RandomForestRegressor(n_estimators=10, random_state=42)  
rf.fit(X_train, Y_train)

In [ ]:
# Make predictions on test
Y_pred2 = rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(Y_test, Y_pred2)
rmse = mse ** (1/2)

rmse_unlogged = mean_squared_error(np.exp(Y_test), np.exp(Y_pred2)) ** (1/2)


print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Root Mean Squared Error (un-logged): {rmse_unlogged:.2f}")

In [ ]:
# RMSE
rmse_unlogged = mean_squared_error(np.exp(Y_test), np.exp(Y_pred2)) ** (1/2)
Y_pred2_train = rf.predict(X_train)
rmse_unlogged_train = mean_squared_error(np.exp(Y_train), np.exp(Y_pred2_train)) ** (1/2)
rmse_unlogged, rmse_unlogged_train

In [ ]:
# Let's visualize this
plt.scatter(np.arange(len(X_test)),np.exp(Y_pred2))
plt.scatter(np.arange(len(X_test)),np.exp(Y_test))

In [ ]:
# Increase n_estimators to 20
rf2 = RandomForestRegressor(n_estimators=20, random_state=42)  
rf2.fit(X_train, Y_train)

In [ ]:
# Make predictions on test
Y_pred2 = rf2.predict(X_test)

# Evaluate the model
mse = mean_squared_error(Y_test, Y_pred2)
rmse = mse ** (1/2)

rmse_unlogged = mean_squared_error(np.exp(Y_test), np.exp(Y_pred2)) ** (1/2)

print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Root Mean Squared Error (un-logged): {rmse_unlogged:.2f}")

In [ ]:
# On Train
rmse_unlogged = mean_squared_error(np.exp(Y_test), np.exp(Y_pred2)) ** (1/2)
Y_pred2_train = rf2.predict(X_train)
rmse_unlogged_train = mean_squared_error(np.exp(Y_train), np.exp(Y_pred2_train)) ** (1/2)
rmse_unlogged, rmse_unlogged_train

In [ ]:
# Let's take another look
plt.scatter(np.arange(len(X_test)),np.exp(Y_pred2))
plt.scatter(np.arange(len(X_test)),np.exp(Y_test))

***Remove data points with valuation above 500 million to avoid outliers***

In [ ]:
# Let's take a look
plt.scatter(np.arange(len(X_train)),np.exp(Y_train))

In [ ]:
# Filter for 'Total Assessed Value' < 250000000
df_new = df_encoded[df_encoded['Total Assessed Value'] < 250000000]

# train-test-split
X = df_new[['Year', 'Assessed Personal Property Value', 'Year Property Built',
       'Number of Bathrooms', 'Number of Bedrooms', 'Number of Rooms',
       'Number of Stories', 'Number of Units', 
       'Incident Counts', 'safe_day_binary', 'safe_night_binary', 'household_size',
       'cleanliness_binary', 'Zipcode_94103', 'Zipcode_94104', 'Zipcode_94105',
       'Zipcode_94107', 'Zipcode_94108', 'Zipcode_94109', 'Zipcode_94110',
       'Zipcode_94111', 'Zipcode_94112', 'Zipcode_94114', 'Zipcode_94115',
       'Zipcode_94116', 'Zipcode_94117', 'Zipcode_94118', 'Zipcode_94121',
       'Zipcode_94122', 'Zipcode_94123', 'Zipcode_94124', 'Zipcode_94127',
       'Zipcode_94131', 'Zipcode_94132', 'Zipcode_94133', 'Zipcode_94134',
       'Zipcode_94158']]
Y = df_new['Log of Total Assessed Value']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .3, random_state = 42)

In [ ]:
# Re-define Model
rf3 = RandomForestRegressor(n_estimators=20, random_state=42)  
rf3.fit(X_train, Y_train)

In [ ]:
# Make predictions on test
Y_pred2 = rf3.predict(X_test)

# Evaluate the model
mse = mean_squared_error(Y_test, Y_pred2)
rmse = mse ** (1/2)

rmse_unlogged = mean_squared_error(np.exp(Y_test), np.exp(Y_pred2)) ** (1/2)

print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Root Mean Squared Error (un-logged): {rmse_unlogged:.2f}")

In [ ]:
# On train
rmse_unlogged = mean_squared_error(np.exp(Y_test), np.exp(Y_pred2)) ** (1/2)
Y_pred2_train = rf3.predict(X_train)
rmse_unlogged_train = mean_squared_error(np.exp(Y_train), np.exp(Y_pred2_train)) ** (1/2)
rmse_unlogged, rmse_unlogged_train

In [ ]:
# Let's take a look again
plt.scatter(np.arange(len(X_test)),np.exp(Y_pred2))
plt.scatter(np.arange(len(X_test)),np.exp(Y_test))
# Looks to be the best performing model

### 3c. Model Results